# Formula Testing: Compare Original, Extended, and Polynomial

This notebook tests the improvements to the learned analytical formula on **CtD** (sparse) and **AeG** (dense) edge types.

## Formulas Tested

### 1. Original (9 parameters)
```
P = α × (u^β × v^γ) / (δ + ε×m + ζ×(u×v)^η + θ×density^κ)
```

### 2. Extended (11 parameters)
```
P = α × (u^β × v^γ) / (δ + ε×m + ζ×(u×v)^η + θ×density^κ + λ×log(u+1) + μ×log(v+1))
```

### 3. Polynomial (9 parameters)
```
P = (α×u×v + β×u + γ×v + δ) / (ε×m + ζ×u×v + η×(u+v) + θ + ι×density)
```

## Improvements Tested
- Multi-start optimization (10 random initializations)
- Configurable L2 regularization

In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Setup paths
repo_dir = Path.cwd().parent
src_dir = repo_dir / 'src'
data_dir = repo_dir / 'data'
results_dir = repo_dir / 'results'

sys.path.append(str(src_dir))

from learned_analytical import LearnedAnalyticalFormula

print("Modules imported successfully!")
print(f"Repository: {repo_dir}")

## Test Configuration

In [ ]:
# Test edge types
test_edge_types = ['CtD', 'AeG']  # Sparse and dense

# Test N values (smaller for faster testing)
test_N_values = [5, 10, 20]

# Formulas to test
formula_types = ['original', 'extended', 'polynomial']

# Configuration
n_random_starts = 10
regularization_lambda = 0.001
convergence_threshold = 0.02
target_metric = 'correlation'
min_metric_value = 0.95

print(f"Testing on edge types: {test_edge_types}")
print(f"Testing N values: {test_N_values}")
print(f"Testing formulas: {formula_types}")
print(f"Multi-start: {n_random_starts} random initializations")
print(f"L2 regularization: λ = {regularization_lambda}")

## Run Tests

In [ ]:
# Storage for results
all_results = []

for edge_type in test_edge_types:
    print(f"\n{'='*80}")
    print(f"TESTING {edge_type}")
    print(f"{'='*80}\n")

    for formula_type in formula_types:
        print(f"\n{'-'*60}")
        print(f"Formula: {formula_type.upper()}")
        print(f"{'-'*60}\n")

        # Initialize learner
        learner = LearnedAnalyticalFormula(
            n_random_starts=n_random_starts,
            regularization_lambda=regularization_lambda,
            formula_type=formula_type
        )

        # Run with limited N values for testing
        try:
            results = learner.find_minimum_permutations(
                graph_name=edge_type,
                data_dir=data_dir,
                results_dir=results_dir,
                N_candidates=test_N_values,
                convergence_threshold=convergence_threshold,
                target_metric=target_metric,
                min_metric_value=min_metric_value
            )

            # Store summary
            all_results.append({
                'edge_type': edge_type,
                'formula': formula_type,
                'N_min': results['N_min'],
                'final_correlation': results['final_metrics']['correlation'],
                'final_mae': results['final_metrics']['mae'],
                'baseline_correlation': results['baseline_metrics']['correlation'],
                'baseline_mae': results['baseline_metrics']['mae'],
                'improvement_correlation': results['final_metrics']['correlation'] - results['baseline_metrics']['correlation'],
                'n_params': len(learner.params)
            })

            print(f"\n✓ SUCCESS")
            print(f"  N_min: {results['N_min']}")
            print(f"  Final correlation: {results['final_metrics']['correlation']:.4f}")
            print(f"  Baseline correlation: {results['baseline_metrics']['correlation']:.4f}")
            print(f"  Improvement: {results['final_metrics']['correlation'] - results['baseline_metrics']['correlation']:+.4f}")

        except Exception as e:
            print(f"\n✗ FAILED: {e}")
            all_results.append({
                'edge_type': edge_type,
                'formula': formula_type,
                'N_min': None,
                'final_correlation': None,
                'final_mae': None,
                'baseline_correlation': None,
                'baseline_mae': None,
                'improvement_correlation': None,
                'n_params': None
            })

# Convert to DataFrame
results_df = pd.DataFrame(all_results)
print(f"\n{'='*80}")
print("ALL TESTS COMPLETE")
print(f"{'='*80}")

## Results Summary

In [ ]:
print("\n" + "="*80)
print("RESULTS SUMMARY")
print("="*80)

for edge_type in test_edge_types:
    print(f"\n{edge_type}:")
    edge_results = results_df[results_df['edge_type'] == edge_type]

    for _, row in edge_results.iterrows():
        if row['final_correlation'] is not None:
            print(f"  {row['formula']:12s}: r={row['final_correlation']:.4f} (baseline={row['baseline_correlation']:.4f}, Δ={row['improvement_correlation']:+.4f})")
        else:
            print(f"  {row['formula']:12s}: FAILED")

# Display full table
print("\nFull Results Table:")
display(results_df)

## Visualization: Formula Comparison

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Correlation comparison
for edge_type, ax in zip(test_edge_types, axes):
    edge_results = results_df[results_df['edge_type'] == edge_type]

    formulas = edge_results['formula'].values
    final_corrs = edge_results['final_correlation'].values
    baseline_corrs = edge_results['baseline_correlation'].values

    x = np.arange(len(formulas))
    width = 0.35

    ax.bar(x - width/2, baseline_corrs, width, label='Current Analytical', alpha=0.7, color='orange')
    ax.bar(x + width/2, final_corrs, width, label='Learned Formula', alpha=0.7, color='green')

    ax.set_xlabel('Formula Type', fontsize=12)
    ax.set_ylabel('Correlation with Empirical', fontsize=12)
    ax.set_title(f'{edge_type} - Correlation Comparison', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(formulas)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

    # Add value labels
    for i, (b, f) in enumerate(zip(baseline_corrs, final_corrs)):
        if b is not None:
            ax.text(i - width/2, b + 0.01, f'{b:.3f}', ha='center', va='bottom', fontsize=9)
        if f is not None:
            ax.text(i + width/2, f + 0.01, f'{f:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig(repo_dir / 'results' / 'formula_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("Saved comparison plot")

## Recommendations

In [ ]:
print("\n" + "="*80)
print("RECOMMENDATIONS")
print("="*80)

# Find best formula for each edge type
for edge_type in test_edge_types:
    edge_results = results_df[results_df['edge_type'] == edge_type]
    best_formula = edge_results.loc[edge_results['final_correlation'].idxmax(), 'formula']
    best_corr = edge_results['final_correlation'].max()
    baseline_corr = edge_results.loc[edge_results['formula'] == best_formula, 'baseline_correlation'].values[0]
    improvement = best_corr - baseline_corr

    print(f"\n{edge_type}:")
    print(f"  Best formula: {best_formula.upper()}")
    print(f"  Correlation: {best_corr:.4f} (baseline: {baseline_corr:.4f})")
    print(f"  Improvement: {improvement:+.4f} ({improvement/baseline_corr*100:+.1f}%)")

    if improvement > 0.01:
        print(f"  ✓ SIGNIFICANT IMPROVEMENT (> 1%)")
    elif improvement > 0:
        print(f"  → Modest improvement")
    else:
        print(f"  ✗ No improvement")

# Overall best formula
avg_by_formula = results_df.groupby('formula')['improvement_correlation'].mean()
overall_best = avg_by_formula.idxmax()
avg_improvement = avg_by_formula.max()

print(f"\n{'='*80}")
print(f"OVERALL BEST FORMULA: {overall_best.upper()}")
print(f"Average improvement: {avg_improvement:+.4f}")
print(f"{'='*80}")

# Save results
results_df.to_csv(repo_dir / 'results' / 'formula_test_results.csv', index=False)
print(f"\nResults saved to: {repo_dir / 'results' / 'formula_test_results.csv'}")